In [41]:
import glob, os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.mask
from matplotlib import pyplot as plt
from rasterio import Affine
from rasterio.warp import reproject, Resampling

In [22]:
#Process the county sowing dataset
current_year = 2001
current_year_str = str(current_year)
os.chdir('/home/heisenberg/my_repo/Data')

df = pd.read_csv('maize.alldat.nolatlon.csv')
IL = (df['state']=='IL')&(df['YEAR']==current_year)
IL_county = df[IL]
col_list = ['FIPS','sday']
IL_county = IL_county[col_list]
IL_county = IL_county.groupby(['FIPS']).mean()
sow_date = list(IL_county['sday'])
del df

#Get all county FIPS in Illinois
county_FIPS = IL_county.index.values
county_FIPS = county_FIPS.tolist()
county_FIPS = list(map(str,county_FIPS))

In [24]:
#Get all counties in IL in the shape file
os.chdir('/home/heisenberg/my_repo/Data/County_Shape')
county_shape = gpd.read_file('CoUS_GCS12.shp')
county_shape = county_shape[county_shape['atlas_stco'].isin(county_FIPS)]
cols = ['atlas_stco','geometry']
county_shape = county_shape[cols]

In [25]:
#Read CDL data and change the CDL data to corn/non-corn
os.chdir('/home/heisenberg/my_repo/Data/CDL')
cdl_name = 'CDL_' + current_year_str + '.tif'
cdl = rasterio.open(cdl_name)
cdl_data = cdl.read(1)
cdl_data[cdl_data != 1] = 0

#Read state shape file
os.chdir('/home/heisenberg/my_repo/Data')
state = gpd.read_file('cb_2017_us_state_500k/')

In [26]:
def mask_by_county(county_shape, corn, i):
    img_clip, trans = rasterio.mask.mask(corn, [county_shape.iloc[i].geometry], crop=True)
    img_clip = img_clip.astype('float32')
    img_clip[img_clip == 0] = np.nan
    index = np.nanmean(img_clip)
    return (img_clip[0,:,:],index)

In [27]:
#Function that returns GCVI index for each county for each GCVI GeoTIFF
def calculate_GCVI(filename, state, cdl, cdl_data, county_shape):
    gcvi = rasterio.open(filename)
    state.crs = gcvi.crs

    img_clip, trans = rasterio.mask.mask(gcvi, [state.iloc[1].geometry], crop=True)
    img_clip = img_clip.astype('float32')
    img_clip[img_clip == 0] = np.nan
    #plt.imshow(img_clip[0,:,:],vmin=0, vmax=800, cmap='Spectral')

    #Project the CDL to GCVI size
    newarr = np.empty(shape=(img_clip.shape[0],
                             img_clip.shape[1],img_clip.shape[2]))
    reproject(
        cdl_data, newarr,
        src_transform = cdl.transform,
        dst_transform = gcvi.transform,
        src_crs = cdl.crs,
        dst_crs = gcvi.crs,
        resampling = Resampling.average)

    #Bigger than 0.5 means there is corn, while smaller than 0.5 means no corn
    is_corn = newarr >= 0.5
    corn_field = is_corn.astype(int)
    corn_field = corn_field.astype(np.float64)

    #Finally mask the corn/non-corn matrix with gcvi to get gcvi indexes
    gcvi_corn = np.multiply(img_clip, corn_field)
    #plt.imshow(gcvi_corn.squeeze())

    #Re-Write the projected GCVI to a new tif file
    os.chdir('/home/heisenberg/my_repo/Data/Output/')
    new_dataset = rasterio.open('filter1.tif', 'w', driver='GTiff',
                                height = corn_field.shape[1], width = corn_field.shape[2],
                                count = 1, dtype = corn_field.dtype,
                                crs = gcvi.crs,
                                transform = gcvi.transform)

    new_dataset.write(gcvi_corn.squeeze(),1)
    new_dataset.close()

    #Change the projection of shape file to cdl file.
    county_shape.crs = cdl.crs
    num_of_county = len(county_shape.index)


    #Read saved gcvi_corn raster file for masking
    test = rasterio.open('filter1.tif')
    #corn_by_county = {}
    index_by_county = {}

    for county in range(num_of_county):
        result = mask_by_county(county_shape,test,county)
        #corn_by_county[county] = result[0]
        index_by_county[county] = result[1]
    os.remove('filter1.tif')
    return index_by_county

In [51]:
#Iterate every tif file in a specific year's gcvi directory.
gcvi_directory = os.fsdecode('/home/heisenberg/my_repo/Data/GCVI/'+ current_year_str)
os.chdir(gcvi_directory)
index = 0
name_index = 2001001
All_GCVI = {}

for infile in sorted(glob.glob(os.path.join('*.tif'))):
    os.chdir(gcvi_directory)
    All_GCVI[index] = calculate_GCVI(infile, state, cdl, cdl_data, county_shape)
    index += 1
    print(infile)
    print("image"+str(index)+" completed \n")
    

print(len(All_GCVI))


GCVI_filled_2001001.tif
image1 completed 

GCVI_filled_2001002.tif
image2 completed 

GCVI_filled_2001003.tif
image3 completed 

GCVI_filled_2001004.tif
image4 completed 

GCVI_filled_2001005.tif
image5 completed 

GCVI_filled_2001006.tif
image6 completed 

GCVI_filled_2001007.tif
image7 completed 

GCVI_filled_2001008.tif
image8 completed 

GCVI_filled_2001009.tif
image9 completed 

GCVI_filled_2001010.tif
image10 completed 

GCVI_filled_2001011.tif
image11 completed 

GCVI_filled_2001012.tif
image12 completed 

GCVI_filled_2001013.tif
image13 completed 

GCVI_filled_2001014.tif
image14 completed 

GCVI_filled_2001015.tif
image15 completed 

GCVI_filled_2001016.tif
image16 completed 

GCVI_filled_2001017.tif
image17 completed 

GCVI_filled_2001018.tif
image18 completed 

GCVI_filled_2001019.tif
image19 completed 

GCVI_filled_2001020.tif
image20 completed 

GCVI_filled_2001021.tif
image21 completed 

GCVI_filled_2001022.tif
image22 completed 

GCVI_filled_2001023.tif
image23 completed

GCVI_filled_2001189.tif
image186 completed 

GCVI_filled_2001190.tif
image187 completed 

GCVI_filled_2001191.tif
image188 completed 

GCVI_filled_2001192.tif
image189 completed 

GCVI_filled_2001193.tif
image190 completed 

GCVI_filled_2001194.tif
image191 completed 

GCVI_filled_2001195.tif
image192 completed 

GCVI_filled_2001196.tif
image193 completed 

GCVI_filled_2001197.tif
image194 completed 

GCVI_filled_2001198.tif
image195 completed 

GCVI_filled_2001199.tif
image196 completed 

GCVI_filled_2001200.tif
image197 completed 

GCVI_filled_2001201.tif
image198 completed 

GCVI_filled_2001202.tif
image199 completed 

GCVI_filled_2001203.tif
image200 completed 

GCVI_filled_2001204.tif
image201 completed 

GCVI_filled_2001205.tif
image202 completed 

GCVI_filled_2001206.tif
image203 completed 

GCVI_filled_2001207.tif
image204 completed 

GCVI_filled_2001208.tif
image205 completed 

GCVI_filled_2001209.tif
image206 completed 

GCVI_filled_2001210.tif
image207 completed 

GCVI_fille

In [66]:
import csv
os.chdir('/home/heisenberg/my_repo/')
with open('mycsvfile.csv', 'wb') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, All_GCVI[0].keys())
    w.writeheader()
    w.writerow(All_GCVI)


TypeError: a bytes-like object is required, not 'str'

In [65]:
y = All_GCVI[0]
y = np.fromiter(y.values(), dtype=float).reshape(1,82)
x = np.linspace(0,81)

print(type(x),type(y))
#plt.plot(x,y)


<class 'numpy.ndarray'> <class 'numpy.ndarray'>
